In [1]:
import numpy as np
import pandas as pd

# Decision Tree Classifier

In this notebook we are going to take you through the how to implement the **[Iterative Deepening 3 (ID3)](https://en.wikipedia.org/wiki/ID3_algorithm)**. 
>**Here we should explain what ID3 is and why it is useful**.

We will start uploading and cleaning a dataset, and then we will implement a series of very useful functions, which in turn will lead us to the implementation of the *DecisionTree* class. This class will define the three most important functions: fit, predict, and print.

So here is the agenda:
1. Dataset Preprocessing
2. Useful Functions Definition
3. DecisionTree class implementation

## 1. Dataset Preprocessing

We will first start by uploading the dataset on which we are going to work. For that purpose we are going to work with *[pandas](https://pandas.pydata.org/)* which is very handy to upload and process dataset files. Let's get started by uploading the file, and taking a look to it.

In [2]:
path = '/home/matlongo/Downloads/homework1/'
dataset = pd.read_csv(path+'dt-data.txt')
dataset

,(Occupied,Price,Music,Location,VIP,Favorite Beer,Enjoy)
0,01: High,Expensive,Loud,Talpiot,No,No,No;
1,02: High,Expensive,Loud,City-Center,Yes,No,Yes;
2,03: Moderate,Normal,Quiet,City-Center,No,Yes,Yes;
3,04: Moderate,Expensive,Quiet,German-Colony,No,No,No;
4,05: Moderate,Expensive,Quiet,German-Colony,Yes,Yes,Yes;
5,06: Moderate,Normal,Quiet,Ein-Karem,No,No,Yes;
6,07: Low,Normal,Quiet,Ein-Karem,No,No,No;
7,08: Moderate,Cheap,Loud,Mahane-Yehuda,No,No,Yes;
8,09: High,Expensive,Loud,City-Center,Yes,Yes,Yes;
9,10: Low,Cheap,Quiet,City-Center,No,No,No;


As we can see, the dataset is quite small (just 22 rows), and only contain 7 features, 6 attributes and one target feature to predict. In our case, the target column to predict is *Enjoy*, and the rest of them are the features we are going to use to train our model, and make predictions.

Besides that, we can also note that some of the attribute names look weird (take *Enjoy)* for instance), and so do some of the attribute values, such as *(Occupied* values have a number before the real value. Therefore, we should do some cleaning to the dataset.

In this block we will: (1) Strip all the fields; (2) Remove punctutation characters from headers; (3) Remove punctuation characters from cells and also the numbers for the first column.

In [3]:
# Here we remove the parenthesis from the header, as well as the extra spaces
dataset.columns = [c.strip().replace('(', '').replace(')', '') for c in dataset.columns]
# Now we will remove the punctuation characters from the Enjoy and Occupied columns
dataset['Enjoy'] = dataset['Enjoy'].apply(lambda enjoy: enjoy[:-1].strip())
dataset['Occupied'] = dataset['Occupied'].apply(lambda occup: occup[4:].strip())
# Finally we remove the spaces from all the cells
dataset = dataset.applymap(lambda s: s.strip())
dataset

,Occupied,Price,Music,Location,VIP,Favorite Beer,Enjoy
0,High,Expensive,Loud,Talpiot,No,No,No
1,High,Expensive,Loud,City-Center,Yes,No,Yes
2,Moderate,Normal,Quiet,City-Center,No,Yes,Yes
3,Moderate,Expensive,Quiet,German-Colony,No,No,No
4,Moderate,Expensive,Quiet,German-Colony,Yes,Yes,Yes
5,Moderate,Normal,Quiet,Ein-Karem,No,No,Yes
6,Low,Normal,Quiet,Ein-Karem,No,No,No
7,Moderate,Cheap,Loud,Mahane-Yehuda,No,No,Yes
8,High,Expensive,Loud,City-Center,Yes,Yes,Yes
9,Low,Cheap,Quiet,City-Center,No,No,No


Great, now everything is set to start implementing the functions.

## 2. Useful Functions Definition

Once the dataset is ready to be used, we will move forwards to implement a couple of functions that form the pieces necessary to implement our final algorithm ID3. 

In order to determine what attribute to use for the split, we are going to use the *Information Gain* metric, which is defined by the following formulae:

$Information\_Gain = Entropy\_Father-Entropy\_child$

So, we need to calculate the Entropy for a given dataset. The Entropy for a probability distribution $\vec{p}$ is calculated with the following formulae:

$Entropy(\vec{p}) = -\sum_{i=1}^{t}p_i*\log(p_i)$

Where *t* is the number of elements in $\vec{p}$.

In our case we want to predict the Entropy for our *target* column. We will call *target* column to the column we will try to estimate, for the shown dataset the target is *Enjoy*. Therefore, to calculate the Entropy for that column, we first need to calculate the probability distribution for each possible value of it:

$\vec{p_{target}} = [\frac{N_{yes}}{N}, \frac{N_{no}}{N}]$

That is, since *Enjoy* only has two possible values (*Yes* and *No*) the probability distribution for it is going to be the number of rows where *Enjoy* is *Yes* ($N_{yes}$) divided by the total number of rows ($N$), and something similar with the number of rows where *Enjoy* is *No*.

We will define a function *get_entropy* that given the dataset, and the target column returns the entropy obtained.

In [8]:
def get_entropy(dataset, target):
    """
    This method returns the entropy for a target column in a given dataset. It basically gets the distribution
    of each class, and based on that it calculates the entropy.
    - dataset: Pandas DataFrame containing all the dataset.
    - target: string representing the dataset's column name for which we want to calculate the entropy.
    
    Returns a float that represents the target column's entropy, in the given dataset.
    """
    # First we check that the column is in the datataset.
    if not(target in dataset):
        raise Exception("The specified target is not present in the given dataset.")
    # First of all we get the number of occurrences for each class in our target column.
    occurrences = dataset[target].value_counts()
    # Now we obtain the probability for each class.
    p_vector = [float(v) / dataset.shape[0] for v in occurrences.values]
    # Finally, we calculate the entropy using the probabilities.
    entropy = -sum([p_i * np.log(p_i) for p_i in p_vector])
    return entropy

Excellent. Now we know how to calculate the entropy for a dataset. As we mentioned at the beginning, *ID3* recursively splits the dataset in sub-datasets based on a particular attribute, until a leaf is obtained, where we will obtain the most popular class. The way to determine what attribute to use to perform the split is by selecting the attribute that gives us the highest Information Gain. So in each level we need to try all the available attributes, perform the split on each of them and calculate the information gain of spliting the dataset using that attribute.

Therefore, the next step is to implement a function that returns the Information Gain we would obtain if we were to split the tree by a given parameter. This function has to split the dataset using all the possible values for that given attribute, calculate the Entropy in all the sub-datasets, and calculate the Entropy of that split using the following formulae:
> Acá va la formula de Lucas

Finally, to calculate the Information Gain we simply subtract the entropy of the split to the Entropy of the previous level.

In [9]:
def get_info_gain_for_attr(dataset, prev_entropy, attr, target):
    """
    This function returns the Information Gain value if we were to select attribute attr to split the dataset
    in different branches.
    - dataset: Pandas DataFrame containing all the dataset.
    - prev_entropy: Entropy from the previous level, necessary to calculate the Information gain. Type float.
    - attr: Attribute's name to be used for calculating the Information gain.
    - target: String representing the dataset's column name for which we want to calculate the entropy and 
    make the predictions.
    
    It returns a float representing the Information gain for spliting the dataset using this attribute. Besides,
    it also returns a dictionary that contains the entropy for each possible value in attr, and the sub-dataset
    corresponding to that value in the column attr.
    """
    # Sanity check
    if not(attr in dataset):
        raise Exception("The specified attr is not present in the given dataset.")
    if not(target in dataset):
        raise Exception("The specified target is not present in the given dataset.")
    
    # First of all we get all the possible values. For example, Yes and No, or High, Moderate and Low.
    possible_values = dataset[attr].drop_duplicates().values
    
    # Now we are going to calculate the entropy for each possible value, and accumulate it in the total_entropy
    # variable. Besides that, we are also going to get the portion of the DataFrame that have the specified value
    # in the attr column, and remove the attr column for that sub-dataset.
    parameters = dict()
    total_entropy = 0
    for i in possible_values:
        # First we get the portion of the dataset that only has the value i.
        dataset_i = dataset.set_index(attr).loc[[i]]
        # Now we calculate the entropy for this sub-dataset.
        entropy_i = get_entropy(dataset_i, target)
        # Finally, we add this entropy to the total entropy for this attribute.
        total_entropy += float(dataset_i.shape[0])/dataset.shape[0]*entropy_i
        parameters[i] = {'dataset': dataset_i, 'entropy': entropy_i}
    return prev_entropy - total_entropy, parameters

In [10]:
class TreeNode:
    def __init__(self, attr_name):
        self.children = None
        self.name = attr_name
        self.class_ = None
    
    def set_children(self, children):
        self.children = children
    
    def set_class(self, class_):
        self.class_ = class_

def get_attr_to_split(dataset, target, prev_entropy):
    
    possible_attributes = set(dataset.columns)-{target}
    target_counts = dataset[target].value_counts() 
    # Cut condition
    if len(possible_attributes)==0 or target_counts.shape[0]==1:
        node = TreeNode("Leaf")
        node.set_class(target_counts.argmax())
        return node
    
    max_gain = -1
    max_params = None
    max_attr = None
    for attr in possible_attributes:
        gain, params = get_info_gain_for_attr(dataset, prev_entropy, attr, target)
        if gain > max_gain:
            max_gain = gain
            max_params = params
            max_attr = attr
    
    node = TreeNode(max_attr)
    children = dict()
    for value, dic in max_params.iteritems():
        children[value] = get_attr_to_split(dic['dataset'], target, dic['entropy'])
    node.set_children(children)
    return node

In [13]:
prev_entropy = get_entropy(dataset, 'Enjoy')
get_attr_to_split(dataset, 'Enjoy', prev_entropy)

Yes    13
No      9
Name: Enjoy, dtype: int64
Yes    4
No     2
Name: Enjoy, dtype: int64
No    1
Name: Enjoy, dtype: int64
No    1
Name: Enjoy, dtype: int64
Yes    3
Name: Enjoy, dtype: int64
Yes    1
Name: Enjoy, dtype: int64
Yes    7
No     2
Name: Enjoy, dtype: int64
No     1
Yes    1
Name: Enjoy, dtype: int64
Yes    1
Name: Enjoy, dtype: int64
No    1
Name: Enjoy, dtype: int64
No     1
Yes    1
Name: Enjoy, dtype: int64
No    1
Name: Enjoy, dtype: int64
Yes    1
Name: Enjoy, dtype: int64
Yes    1
Name: Enjoy, dtype: int64
Yes    2
Name: Enjoy, dtype: int64
Yes    2
Name: Enjoy, dtype: int64
No     5
Yes    2
Name: Enjoy, dtype: int64
No    1
Name: Enjoy, dtype: int64
No     2
Yes    1
Name: Enjoy, dtype: int64
No    1
Name: Enjoy, dtype: int64
No     1
Yes    1
Name: Enjoy, dtype: int64
No     1
Yes    1
Name: Enjoy, dtype: int64
No     1
Yes    1
Name: Enjoy, dtype: int64
No     1
Yes    1
Name: Enjoy, dtype: int64
No    1
Name: Enjoy, dtype: int64
No     1
Yes    1
Name: Enjoy, 

<__main__.TreeNode instance at 0x7fa5e1318b00>

In [ ]:
tree = TreeNode()
tree.fit(dataset, target)

In [ ]:
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
cls = tree.DecisionTreeClassifier(criterion='entropy')
one_hot = OneHotEncoder()
pipeline = Pipeline([('encoder', one_hot), ('classifier', cls)])
encoder = LabelEncoder()

X = np.array([encoder.fit_transform(column) for column in dataset.drop('Enjoy', 1).values.T]).T

In [ ]:
pipeline.fit(X, dataset['Enjoy'].values)

In [ ]:
tree.export_graphviz(pipeline.named_steps.classifier, out_file='tree.dot')  

In [ ]:
!dot -Tpng tree.dot -o tree.png

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
plt.figure(figsize=(12,18))
plt.axis('off')
img=mpimg.imread('tree.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
columns = dataset.drop('Enjoy', 1).columns
ax = 0
for j in range(len(columns)):
    column = columns[j]
    encoder.fit_transform(dataset[column])
    #print "x_"+str(j)+": "+column
    for i in range(len(encoder.classes_)):
        print "x_"+str(ax)+": "+column+"_"+encoder.classes_[i]
        ax += 1